## Build out the functionality to automate measure creation for a tabular model

#### Currently the way this is setup is:
1. Manually copy the file from the repo into the local location
2. Change the variables in the CONFIGURATION step
3. Run the script
4. Close the model file in Visual Studio or anywhere else
5. Manually copy the file back to the repo location
6. Open the model in Visual Studio to check for errors

#### TODO (Future enhancements): 
- Add in configuration to point to the repo where the model exists
    - How to handle if the user has the file open?
    - Copy the model to a local location, make changes to a new file, and paste back into the repo
- Add in configuration file for all the base measures that the user wants to process

#### This should be able to modify the .bim file and create the following metrics given a base metric:
Current Year:
- PD
- WTD
- MTD
- QTD
- YTD

Last Year:

- PDPY
- WTDPY
- MTDPY
- QTDPY
- YTDPY

Growth:
- Day Over Day Growth %
- WTD Growth %
- MTD Growth % (Date)
- MTD Growth % (Day)
- QTD Growth %
- YTD Growth %

In [1]:
import json
import os

In [2]:
def getMeasures(baseMeasure,measureFormat,measureAccuracy,hiddenMeasure=True):
    if measureAccuracy > 0:
        accuracyString = "."+measureAccuracy*"0"
    else:
        accuracyString=""
    
    if measureFormat == 'Currency':    
        formatString = "\\$#,0"+accuracyString+";(\\$#,0"+accuracyString+");\\$#,0"+accuracyString
        annotationValueString = "<Format Format=\"Currency\" Accuracy=\""+str(measureAccuracy)+"\"><Currency LCID=\"1033\" DisplayName=\"$ English (United States)\" Symbol=\"$\" PositivePattern=\"0\" NegativePattern=\"0\" /></Format>"
        growthCalculation = "DIVIDE((cur-prev),prev)"
        #default growth format is a 2 decimal place percent
        growthFormatString = "0.00%;-0.00%;0.00%"
        
    elif measureFormat == 'Percent':
        formatString = "0"+accuracyString+"%;-0"+accuracyString+"%;0"+accuracyString+"%"
        annotationValueString = ""
        #change the growth to be a straight difference between the timeframes and not a true growth metric
        growthCalculation = "(cur-prev)"
        growthFormatString = formatString
    else:
        formatString = "0"
        annotationValueString = ""
        growthCalculation = "DIVIDE((cur-prev),prev)"
        #default growth format is a 2 decimal place percent
        growthFormatString = "0.00%;-0.00%;0.00%"
        
    #define the measures we want to add to the model

    mtdMeasure = {
            "name": baseMeasure+" MTD",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[MTD Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[MTD End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }
    qtdMeasure = {
            "name": baseMeasure+" QTD",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[QTD Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[QTD End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }
    ytdMeasure = {
            "name": baseMeasure+" YTD",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[YTD Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[YTD End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }
    
    mtdDatePriorYearMeasure = {
            "name": baseMeasure+" MTDPY",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[MTDPY Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[MTDPY End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }

    qtdPriorYearMeasure = {
            "name": baseMeasure+" QTDPY",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[QTDPY Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[QTDPY End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }
    ytdPriorYearMeasure = {
            "name": baseMeasure+" YTDPY",
            "expression": [
              " ",
              "var startDate = LOOKUPVALUE('Calendar'[YTDPY Start],'Calendar'[Date],[Reference Date])",
              "var endDate= LOOKUPVALUE('Calendar'[YTDPY End],'Calendar'[Date],[Reference Date])",
              "return ",
              "CALCULATE(["+baseMeasure+"],DATESBETWEEN('Calendar'[Date],startDate,endDate))"
            ],
            "formatString": formatString,
            "isHidden": hiddenMeasure,
            "annotations": [
                  {
                    "name": "Format",
                    "value": annotationValueString
                  }
                ]
          }
    mtdDateGrowthMeasure = {
            "name": baseMeasure+" MTD Growth %",
            "expression": [
              " ",
              "var cur = ["+baseMeasure+" MTD]",
              "var prev = ["+baseMeasure+" MTDPY]",
              "return",
              growthCalculation
            ],
            "formatString": growthFormatString,
            "isHidden": hiddenMeasure
          }
    qtdGrowthMeasure = {
            "name": baseMeasure+" QTD Growth %",
            "expression": [
              " ",
              "var cur = ["+baseMeasure+" QTD]",
              "var prev = ["+baseMeasure+" QTDPY]",
              "return",
              growthCalculation
            ],
            "formatString": growthFormatString,
            "isHidden": hiddenMeasure
          }
    ytdGrowthMeasure = {
            "name": baseMeasure+" YTD Growth %",
            "expression": [
              " ",
              "var cur = ["+baseMeasure+" YTD]",
              "var prev = ["+baseMeasure+" YTDPY]",
              "return",
              growthCalculation
            ],
            "formatString": growthFormatString,
            "isHidden": hiddenMeasure
          }
    
    retMeasures = [mtdMeasure
                  ,qtdMeasure
                  ,ytdMeasure
                  ,mtdDatePriorYearMeasure
                  ,qtdPriorYearMeasure
                  ,ytdPriorYearMeasure
                  ,mtdDateGrowthMeasure
                  ,qtdGrowthMeasure
                  ,ytdGrowthMeasure]
    
    return retMeasures

In [3]:
#Work with the base model file
#The structure is:
#tables
#   name
#   columns
#   partitions
#   measures

def addMeasures(jsonObj,table,baseMeasure,measureFormat,measureAccuracy,):
    measuresToAdd = getMeasures(baseMeasure,measureFormat,measureAccuracy,False)
    
    tables = jsonObj['model']['tables']
        
    #Find the table you want to add the measure to
    tableFound = False
    for t in tables:
        if t['name'] == table:
            #Add the measure you want to 
            for m in measuresToAdd:
                measures = t['measures'].append(m)
            
            #indicate that we've found the table
            tableFound=True
            break
            
    if not tableFound:
        print('Error: destination table for measure not found')

In [4]:
#CONFIGURATION VARIABLES

#C:\Users\JScherbring\source\repos\TestTabular\TestTabular
sourceDirectory = os.path.join('C:\\','Users','Jscherbring','source','repos','TestTabular','TestTabular')
#'type' - Currently only support "Numeric" and "Currency"
#'accuracy' - determines the number of decimal points

measureDictionary = [
                    {'tableName':'Orders',
                      'baseMeasureName':'Number of Orders',
                      'format': {'type':'Numeric', 
                                 'accuracy':0   
                                }
                     },{'tableName':'Orders',
                      'baseMeasureName':'Sales $',
                      'format': {'type':'Currency', 
                                 'accuracy':0   
                                }
                     },{'tableName':'Orders',
                      'baseMeasureName':'Number of Customers',
                      'format': {'type':'Numeric', 
                                 'accuracy':0   
                                }
                     }
                      
                     ]
inputModelFileName = 'Model.bim'
modelOutputFileName = 'Model.bim'
modelFileIn = os.path.join(sourceDirectory,inputModelFileName)
modelFileOut = os.path.join(sourceDirectory,modelOutputFileName)

In [5]:
with open(modelFileIn) as f:
    obj = json.load(f)

for m in measureDictionary:
    addMeasures(obj,m['tableName'],m['baseMeasureName'],m['format']['type'],m['format']['accuracy'])
    
# EDIT: writing the modified json to file
with open(modelFileOut, "w") as fout:
    json.dump(obj, fout, indent="  ")
